In [6]:
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from myutils import *
import random

In [3]:
# network definition
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
# dataset
fashion_mnist_data = torchvision.datasets.FashionMNIST(
    './fashion-mnist',
    transform=torchvision.transforms.ToTensor(),
    download=True)

data_loader = torch.utils.data.DataLoader(
    dataset=fashion_mnist_data,
    batch_size=16,
    shuffle=True)

fashion_mnist_data_test = torchvision.datasets.FashionMNIST(
    './fashion-mnist',
    transform=torchvision.transforms.ToTensor(),
    train=False,
    download=True)

data_loader_test = torch.utils.data.DataLoader(
    dataset=fashion_mnist_data_test,
    batch_size=16,
    shuffle=True)

In [5]:
# start learning
using_cuda = torch.cuda.is_available()
net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.NLLLoss()
if using_cuda:
    net.cuda()
    criterion.cuda()

accuracies = []
epochs = 40

In [7]:
for i in range(epochs):
    for batch, labels in data_loader:
        if using_cuda:
            x = Variable(batch.cuda())
            y = Variable(labels.cuda())
        else:
            x = Variable(batch)
            y = Variable(labels)
        optimizer.zero_grad()
        output = net(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        

    # test
    n_true = 0
    for batch, labels in data_loader_test:
        if using_cuda:
            output = net(Variable(batch.cuda()))
        else:
            output = net(Variable(batch))
        
        _, predicted = torch.max(output.data, 1)
        if using_cuda:
            y_predicted = predicted.cpu().numpy()
        else:
            y_predicted = predicted.numpy()
        print(predicted,labels)
        n_true += np.sum(y_predicted == labels.numpy())

    total = len(fashion_mnist_data_test)
    accuracy = 100 * n_true / total
    print('epoch: {0}, accuracy: {1}'.format(i, (int)(accuracy)))
    accuracies.append(accuracy)

print(accuracies)



NameError: name 'data_loader' is not defined

In [97]:
src_domains, (X_test, y_test) = load_rotated_mnist()
X_list = []
y_list = []
domain = []
for d in range(0, len(src_domains)):
    X, y = src_domains[d]
    #n_labels = len(np.unique(y))  # 分類クラスの数 = 10
    #y = np.eye(n_labels)[y]       # one hot表現に変換
    X_list.append(X)
    y_list.append(y)#0,15,30,45,60 
    domain.append(d)

#train_dataset = data.TensorDataset(torch.tensor(X_list),torch.tensor(y_list))
#test_dataset = data.TensorDataset(torch.tensor(X_test),torch.tensor(y_test))
X_list, y_list, X_test, y_test = torch.tensor(X_list).float(), torch.tensor(y_list).long(),torch.tensor(X_test).float(),torch.tensor(y_test).long()
X_list, y_list = X_list.view(5000,256), y_list.view(5000,1)
train_dataset = torch.utils.data.TensorDataset(X_list, y_list)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
data_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=500,
    shuffle=True)
data_loader_test = torch.utils.data.DataLoader(
    dataset= test_dataset,
    batch_size = 512,
    shuffle = True)


In [17]:
src_domains, (X_test, y_test) = load_rotated_mnist()
X_list = []
y_list = []
domain = []
for d in range(0, len(src_domains)):
    X, y = src_domains[d]
    #n_labels = len(np.unique(y))  # 分類クラスの数 = 10
    #y = np.eye(n_labels)[y]       # one hot表現に変換
    X_list.append(X)
    y_list.append(y)#0,15,30,45,60 
    domain.append(d)

#train_dataset = data.TensorDataset(torch.tensor(X_list),torch.tensor(y_list))
#test_dataset = data.TensorDataset(torch.tensor(X_test),torch.tensor(y_test))
X_list, y_list, X_test, y_test = torch.tensor(X_list).float(), torch.tensor(y_list).long(),torch.tensor(X_test).float(),torch.tensor(y_test).long()

r_dom = list(range(10))
data_r_seet = list(range(1000))
dom_data_seet = []
#for i in domain:#各ドメインのデータの並び替え
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

data_loader_test = torch.utils.data.DataLoader(
    dataset= test_dataset,
    batch_size = 512,
    shuffle = True)


In [18]:
# network definition
class Net(nn.Module):

    def __init__(self,n_features,n_c):
        super(Net, self).__init__()
        self.n_features = n_features
        self.n_c = n_c 
        self.h_func = nn.Sequential(
                      nn.Linear(self.n_features,100),
                      nn.ReLU(),
                      nn.Linear(100,self.n_c)
                    )
        
    def forward(self, x):
        h = self.h_func(x)
        h = nn.functional.log_softmax(h,dim=1)
        return h

In [19]:
# start learning
using_cuda = torch.cuda.is_available()
net = Net(256,10)
optimizer = optim.Adam(net.parameters(), lr=0.001)
criterion = nn.NLLLoss()
if using_cuda:
    net.cuda()
    criterion.cuda()

accuracies = []
epochs = 300

In [21]:
 for i in range(epochs):  
    r_dom = list(range(10))
    data_r_seet = list(range(1000))
    random.shuffle(r_dom)#DomainNumberをシャッフル
    random.shuffle(data_r_seet)#DataNumberをシャッフル 
    for i in r_dom:#Batch Learining
        
        seet = data_r_seet[100*i:100*(1+i)]#100飛ばし

        X_batch = Variable(X_list[:,seet,:].cuda())#[domain:5, samples:100,  dim:256]
        y_batch = Variable(y_list[:,seet].flatten().cuda())#[5,100]=>[500] 
        optimizer.zero_grad()
        output = net(X_batch)
        output = output.view(500,10)

        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step() 
    n_true = 0    
    for batch, labels in data_loader_test:
        if using_cuda:
            output = net(Variable(batch.cuda()))
        else:
            output = net(Variable(batch))
        
        _, predicted = torch.max(output.data, 1)
        if using_cuda:
            y_predicted = predicted.cpu().numpy()
        else:
            y_predicted = predicted.numpy()
        print(predicted,labels)
        n_true += np.sum(y_predicted == labels.numpy())

    total = 1000
    accuracy = 100 * n_true / total
    print('epoch: {0}, accuracy: {1}'.format(i, (int)(accuracy)))
    accuracies.append(accuracy)


print(accuracies) 

tensor([1, 8, 8, 1, 3, 2, 1, 3, 9, 0, 4, 2, 2, 3, 7, 3, 0, 6, 0, 2, 8, 8, 5, 8,
        0, 8, 8, 1, 0, 2, 8, 0, 2, 5, 5, 2, 3, 8, 7, 2, 5, 6, 9, 0, 7, 2, 1, 2,
        0, 1, 6, 9, 1, 2, 0, 2, 8, 4, 2, 4, 8, 3, 8, 1, 8, 9, 9, 2, 4, 1, 4, 8,
        8, 8, 4, 0, 9, 0, 8, 8, 6, 2, 1, 6, 8, 6, 2, 2, 2, 5, 5, 5, 1, 2, 5, 0,
        8, 2, 2, 2, 2, 0, 6, 2, 8, 8, 7, 1, 1, 2, 4, 8, 6, 6, 9, 2, 4, 4, 0, 7,
        8, 9, 5, 1, 5, 9, 5, 0, 0, 6, 2, 0, 9, 9, 0, 7, 2, 3, 0, 4, 3, 8, 8, 7,
        3, 1, 2, 4, 7, 9, 4, 6, 2, 8, 2, 1, 1, 1, 8, 3, 6, 3, 8, 6, 6, 2, 1, 3,
        5, 7, 0, 4, 8, 3, 4, 4, 3, 6, 6, 8, 3, 0, 6, 5, 6, 2, 4, 4, 8, 0, 0, 2,
        1, 1, 4, 5, 8, 2, 7, 9, 9, 0, 3, 2, 2, 3, 2, 1, 3, 0, 4, 2, 2, 8, 9, 0,
        3, 8, 4, 2, 3, 2, 9, 9, 1, 7, 0, 8, 1, 4, 6, 1, 0, 3, 9, 0, 3, 2, 0, 8,
        5, 8, 4, 4, 0, 8, 9, 4, 3, 1, 0, 9, 2, 4, 4, 2, 6, 5, 6, 4, 1, 6, 9, 7,
        7, 9, 3, 4, 1, 0, 8, 3, 1, 1, 9, 8, 3, 9, 8, 2, 2, 1, 1, 5, 1, 3, 5, 9,
        1, 6, 8, 7, 6, 3, 2, 9, 0, 2, 5,

tensor([3, 2, 3, 8, 1, 3, 8, 8, 2, 3, 4, 3, 0, 8, 2, 1, 2, 1, 0, 8, 7, 9, 5, 8,
        2, 3, 8, 0, 8, 4, 6, 1, 2, 3, 1, 2, 1, 1, 0, 8, 8, 8, 2, 0, 2, 3, 3, 9,
        6, 1, 0, 1, 2, 8, 1, 9, 0, 6, 7, 7, 4, 0, 3, 2, 2, 4, 0, 2, 0, 8, 6, 1,
        0, 9, 8, 3, 8, 7, 4, 1, 7, 1, 4, 0, 2, 6, 9, 5, 4, 2, 2, 2, 2, 1, 2, 9,
        3, 2, 9, 1, 3, 2, 4, 5, 2, 2, 2, 9, 1, 3, 3, 0, 2, 8, 0, 9, 1, 3, 1, 0,
        2, 5, 1, 2, 9, 8, 6, 1, 8, 8, 8, 6, 2, 7, 0, 7, 5, 7, 0, 3, 2, 6, 3, 3,
        1, 2, 6, 5, 2, 3, 2, 0, 6, 6, 0, 1, 2, 5, 0, 3, 2, 1, 2, 9, 8, 3, 4, 8,
        6, 2, 9, 2, 8, 2, 2, 8, 3, 3, 2, 2, 2, 9, 3, 1, 1, 8, 4, 2, 2, 8, 6, 0,
        0, 2, 8, 1, 9, 6, 9, 6, 0, 1, 5, 7, 0, 2, 7, 7, 2, 6, 2, 3, 2, 1, 6, 1,
        3, 3, 8, 2, 2, 0, 0, 8, 9, 0, 2, 1, 3, 5, 3, 7, 2, 8, 3, 4, 3, 3, 1, 8,
        3, 7, 5, 4, 1, 9, 8, 2, 6, 9, 2, 4, 5, 5, 0, 2, 2, 2, 8, 6, 1, 3, 1, 1,
        1, 5, 3, 9, 6, 2, 8, 3, 8, 6, 1, 0, 8, 2, 3, 8, 1, 9, 2, 2, 0, 7, 6, 2,
        3, 4, 4, 6, 9, 0, 9, 7, 1, 9, 5,

tensor([8, 6, 5, 6, 8, 0, 1, 1, 8, 9, 0, 1, 8, 0, 3, 0, 6, 2, 3, 4, 0, 8, 2, 8,
        8, 3, 0, 1, 2, 5, 2, 4, 4, 1, 1, 8, 6, 1, 0, 5, 5, 8, 7, 7, 5, 2, 4, 2,
        8, 4, 1, 5, 9, 2, 1, 2, 2, 8, 0, 1, 4, 4, 8, 3, 1, 6, 0, 3, 7, 3, 0, 2,
        2, 0, 4, 0, 8, 4, 5, 6, 3, 1, 6, 2, 2, 8, 9, 9, 3, 3, 2, 2, 2, 8, 3, 4,
        4, 1, 0, 9, 5, 2, 4, 7, 5, 7, 4, 4, 0, 8, 0, 0, 2, 2, 4, 3, 5, 3, 1, 8,
        4, 3, 2, 8, 1, 9, 5, 5, 0, 5, 0, 1, 0, 2, 1, 2, 1, 8, 5, 2, 9, 0, 2, 2,
        3, 3, 2, 2, 5, 0, 2, 9, 5, 3, 3, 3, 8, 1, 2, 9, 0, 2, 6, 3, 1, 2, 2, 2,
        5, 8, 7, 2, 8, 2, 3, 8, 2, 6, 1, 5, 2, 6, 8, 2, 9, 6, 2, 1, 4, 8, 7, 2,
        4, 3, 8, 8, 9, 2, 5, 2, 0, 4, 8, 3, 2, 1, 1, 8, 2, 5, 6, 2, 6, 5, 3, 8,
        8, 1, 0, 5, 3, 3, 2, 3, 7, 5, 9, 8, 3, 8, 6, 4, 1, 4, 6, 2, 7, 2, 2, 1,
        0, 8, 7, 7, 9, 2, 1, 4, 2, 4, 6, 8, 4, 0, 6, 2, 2, 3, 5, 3, 9, 3, 4, 8,
        3, 3, 0, 2, 2, 8, 3, 2, 2, 8, 6, 2, 9, 4, 0, 8, 1, 5, 8, 8, 0, 8, 2, 4,
        6, 5, 0, 2, 5, 2, 7, 1, 8, 3, 9,

tensor([2, 5, 2, 3, 9, 2, 2, 5, 8, 8, 2, 2, 2, 0, 8, 8, 7, 0, 6, 9, 2, 0, 3, 0,
        6, 1, 2, 2, 2, 4, 5, 1, 2, 8, 8, 7, 2, 8, 1, 4, 9, 5, 6, 2, 9, 8, 4, 2,
        3, 1, 0, 3, 1, 3, 7, 5, 5, 4, 3, 0, 8, 2, 8, 2, 7, 1, 4, 0, 4, 3, 1, 8,
        2, 4, 5, 3, 5, 8, 2, 0, 3, 7, 8, 3, 3, 7, 1, 2, 0, 8, 8, 8, 1, 0, 9, 3,
        2, 4, 1, 6, 7, 3, 3, 1, 1, 3, 8, 3, 4, 5, 7, 5, 1, 2, 9, 0, 2, 2, 4, 1,
        2, 8, 0, 9, 8, 6, 8, 2, 0, 6, 8, 2, 4, 1, 0, 3, 8, 7, 1, 2, 4, 3, 8, 2,
        5, 5, 3, 0, 0, 1, 9, 4, 2, 2, 2, 1, 0, 5, 8, 2, 7, 4, 1, 8, 2, 3, 2, 5,
        3, 1, 3, 1, 3, 2, 0, 0, 5, 2, 8, 7, 4, 9, 8, 1, 2, 9, 6, 9, 3, 8, 1, 2,
        0, 5, 9, 2, 2, 6, 0, 2, 0, 2, 3, 4, 3, 6, 9, 5, 3, 9, 9, 4, 4, 2, 2, 9,
        1, 1, 5, 0, 2, 9, 3, 2, 6, 6, 1, 0, 0, 0, 6, 0, 0, 7, 5, 6, 7, 5, 4, 1,
        3, 2, 8, 3, 5, 0, 0, 2, 4, 2, 2, 5, 1, 9, 4, 3, 3, 2, 3, 2, 1, 4, 1, 8,
        0, 9, 5, 2, 0, 6, 0, 8, 8, 3, 6, 0, 0, 0, 7, 7, 2, 0, 3, 9, 2, 1, 3, 7,
        0, 4, 1, 7, 8, 8, 2, 9, 8, 2, 6,

tensor([1, 1, 8, 8, 1, 0, 6, 8, 3, 3, 2, 2, 8, 2, 2, 2, 8, 3, 7, 0, 2, 2, 8, 3,
        5, 5, 2, 1, 2, 3, 7, 8, 1, 3, 3, 2, 8, 0, 4, 4, 5, 4, 9, 9, 0, 8, 6, 1,
        0, 5, 0, 3, 5, 6, 8, 1, 7, 1, 8, 6, 9, 7, 1, 0, 4, 6, 0, 0, 6, 3, 8, 0,
        6, 1, 3, 0, 0, 2, 3, 9, 5, 6, 1, 2, 8, 7, 0, 6, 8, 2, 4, 9, 9, 4, 3, 1,
        5, 4, 0, 2, 2, 0, 4, 2, 6, 7, 2, 2, 2, 9, 9, 8, 8, 0, 8, 4, 6, 3, 4, 5,
        2, 9, 5, 1, 4, 3, 8, 5, 2, 0, 2, 9, 2, 2, 4, 0, 2, 0, 2, 0, 0, 8, 0, 1,
        0, 9, 6, 2, 8, 0, 9, 2, 2, 1, 1, 4, 2, 6, 0, 6, 2, 5, 3, 9, 1, 0, 9, 6,
        2, 6, 5, 0, 6, 0, 8, 6, 7, 8, 3, 2, 8, 0, 5, 2, 2, 9, 6, 1, 8, 6, 9, 3,
        8, 2, 8, 6, 6, 4, 2, 5, 1, 0, 8, 1, 2, 8, 8, 6, 3, 2, 5, 6, 2, 4, 4, 6,
        6, 3, 2, 2, 0, 6, 1, 7, 3, 6, 5, 5, 0, 0, 4, 5, 5, 7, 2, 1, 8, 2, 0, 2,
        3, 4, 6, 8, 0, 3, 1, 9, 4, 2, 1, 1, 8, 1, 9, 3, 7, 2, 4, 7, 8, 8, 5, 3,
        2, 1, 0, 2, 4, 2, 0, 9, 1, 4, 0, 2, 3, 5, 9, 0, 6, 1, 8, 2, 5, 7, 2, 1,
        9, 1, 2, 4, 8, 0, 0, 3, 9, 8, 2,

...

tensor([4, 8, 4, 5, 5, 1, 6, 9, 4, 0, 9, 3, 2, 7, 4, 6, 6, 8, 4, 8, 3, 3, 3, 2,
        2, 2, 6, 2, 5, 6, 1, 8, 7, 8, 6, 4, 3, 9, 8, 2, 9, 0, 7, 2, 0, 9, 0, 5,
        6, 2, 6, 9, 2, 8, 1, 8, 8, 6, 4, 5, 3, 4, 4, 0, 8, 5, 0, 8, 7, 6, 8, 6,
        1, 1, 2, 5, 8, 8, 4, 6, 8, 2, 6, 2, 6, 1, 3, 8, 2, 7, 6, 6, 4, 0, 3, 4,
        4, 1, 4, 0, 0, 0, 5, 1, 0, 2, 2, 6, 6, 0, 3, 5, 8, 7, 5, 6, 2, 7, 7, 9,
        8, 2, 3, 4, 3, 1, 2, 1, 2, 8, 0, 0, 2, 7, 8, 8, 0, 0, 7, 4, 1, 1, 5, 4,
        0, 4, 1, 7, 1, 9, 0, 0, 2, 8, 6, 2, 8, 4, 7, 1, 3, 8, 0, 3, 8, 3, 7, 0,
        6, 0, 2, 0, 9, 0, 7, 2, 2, 5, 2, 9, 1, 2, 2, 2, 2, 1, 8, 5, 2, 5, 6, 7,
        8, 5, 8, 0, 6, 5, 5, 3, 2, 6, 8, 9, 1, 1, 9, 3, 8, 3, 6, 4, 6, 8, 3, 8,
        5, 2, 2, 5, 5, 5, 3, 5, 5, 6, 8, 2, 1, 1, 2, 7, 2, 3, 0, 0, 2, 0, 4, 1,
        0, 6, 2, 5, 0, 1, 0, 0, 1, 2, 0, 1, 3, 8, 9, 1, 8, 2, 4, 2, 7, 9, 8, 5,
        8, 3, 9, 2, 1, 4, 6, 2, 2, 3, 0, 8, 2, 6, 8, 8, 0, 6, 8, 4, 0, 2, 1, 2,
        6, 3, 1, 9, 7, 5, 5, 4, 4, 2, 7,

tensor([0, 4, 6, 7, 4, 2, 2, 8, 3, 0, 9, 4, 7, 0, 1, 0, 9, 2, 1, 3, 9, 2, 6, 7,
        0, 6, 9, 8, 2, 0, 2, 2, 2, 8, 5, 0, 4, 9, 0, 1, 0, 8, 3, 2, 3, 6, 6, 8,
        0, 4, 3, 2, 4, 1, 4, 1, 6, 6, 2, 9, 3, 2, 0, 2, 3, 6, 8, 0, 5, 8, 2, 8,
        3, 7, 2, 2, 8, 8, 2, 4, 5, 7, 2, 4, 2, 6, 1, 1, 0, 5, 6, 1, 4, 2, 1, 7,
        6, 6, 8, 7, 8, 1, 8, 3, 9, 7, 2, 3, 0, 4, 2, 6, 4, 6, 9, 9, 0, 0, 3, 6,
        8, 1, 7, 2, 8, 5, 5, 5, 7, 9, 0, 2, 0, 8, 1, 5, 2, 2, 5, 2, 5, 4, 6, 2,
        5, 5, 4, 2, 8, 1, 5, 3, 1, 4, 1, 9, 2, 1, 2, 2, 9, 4, 7, 2, 0, 4, 1, 0,
        8, 2, 1, 1, 6, 2, 5, 1, 3, 2, 3, 4, 9, 2, 1, 5, 2, 3, 2, 2, 2, 0, 7, 8,
        6, 2, 8, 0, 4, 4, 8, 7, 6, 2, 2, 9, 2, 8, 2, 4, 0, 8, 0, 0, 8, 4, 1, 1,
        8, 5, 8, 6, 3, 6, 6, 2, 6, 1, 1, 4, 7, 2, 5, 6, 3, 6, 2, 5, 2, 5, 4, 2,
        2, 7, 6, 1, 3, 4, 4, 2, 7, 4, 8, 0, 4, 5, 7, 1, 3, 2, 0, 8, 2, 1, 7, 1,
        5, 4, 0, 3, 0, 1, 0, 2, 2, 6, 0, 2, 2, 2, 4, 8, 7, 0, 3, 3, 3, 8, 3, 0,
        2, 3, 2, 8, 3, 8, 4, 6, 2, 0, 3,

tensor([0, 2, 5, 5, 5, 2, 6, 4, 3, 8, 9, 5, 1, 3, 4, 0, 5, 8, 5, 1, 1, 8, 4, 0,
        2, 1, 4, 2, 3, 6, 2, 5, 2, 0, 2, 0, 8, 5, 0, 1, 8, 6, 2, 8, 7, 2, 3, 2,
        2, 2, 6, 8, 8, 6, 0, 9, 8, 5, 2, 4, 1, 3, 0, 3, 0, 9, 1, 6, 0, 6, 8, 0,
        4, 8, 6, 4, 5, 6, 3, 9, 2, 1, 1, 8, 1, 3, 2, 5, 8, 6, 3, 2, 6, 7, 5, 2,
        7, 5, 5, 8, 3, 1, 9, 2, 0, 2, 7, 1, 3, 7, 2, 2, 0, 1, 2, 2, 2, 6, 8, 4,
        9, 2, 0, 5, 8, 2, 0, 2, 5, 6, 7, 7, 6, 6, 6, 6, 6, 5, 0, 5, 7, 4, 3, 0,
        2, 2, 8, 5, 6, 2, 9, 2, 9, 3, 1, 3, 6, 0, 1, 8, 2, 4, 1, 2, 2, 4, 0, 0,
        5, 6, 0, 5, 3, 4, 3, 9, 2, 2, 1, 3, 0, 3, 1, 2, 2, 9, 4, 2, 5, 9, 1, 3,
        1, 6, 1, 2, 0, 9, 0, 9, 8, 6, 8, 3, 2, 5, 9, 2, 3, 3, 6, 1, 2, 2, 0, 4,
        4, 7, 4, 5, 8, 3, 3, 6, 1, 5, 8, 8, 2, 9, 2, 2, 8, 6, 8, 8, 2, 2, 0, 2,
        1, 0, 8, 5, 6, 4, 3, 9, 2, 2, 8, 2, 8, 3, 9, 6, 2, 2, 4, 8, 0, 8, 8, 1,
        4, 0, 8, 1, 6, 6, 1, 2, 1, 0, 7, 7, 7, 5, 8, 2, 2, 6, 6, 3, 9, 7, 2, 2,
        0, 0, 6, 8, 0, 1, 2, 0, 6, 3, 2,

tensor([8, 9, 9, 3, 2, 3, 2, 8, 6, 8, 7, 8, 8, 2, 2, 3, 2, 5, 5, 1, 6, 0, 7, 2,
        1, 2, 1, 2, 0, 4, 6, 6, 0, 3, 0, 6, 9, 3, 2, 1, 0, 8, 2, 3, 0, 3, 2, 0,
        3, 9, 4, 9, 2, 3, 3, 2, 3, 0, 2, 0, 2, 8, 3, 1, 2, 8, 3, 8, 0, 6, 2, 8,
        3, 4, 8, 2, 5, 3, 8, 7, 6, 9, 2, 2, 7, 0, 6, 9, 3, 7, 0, 0, 1, 0, 2, 1,
        4, 2, 9, 7, 4, 5, 9, 3, 4, 5, 0, 1, 3, 4, 9, 6, 0, 0, 0, 8, 0, 8, 8, 2,
        1, 5, 2, 2, 1, 1, 2, 4, 2, 8, 1, 4, 3, 3, 8, 2, 2, 9, 1, 6, 4, 2, 7, 8,
        8, 5, 5, 1, 6, 2, 3, 4, 6, 8, 0, 1, 3, 9, 8, 5, 3, 3, 4, 2, 4, 3, 8, 2,
        8, 8, 8, 7, 3, 7, 2, 8, 1, 5, 1, 8, 1, 7, 9, 8, 3, 2, 3, 4, 2, 0, 2, 3,
        8, 6, 5, 5, 2, 1, 5, 1, 7, 4, 8, 1, 5, 2, 5, 6, 2, 7, 6, 5, 2, 2, 4, 0,
        2, 0, 8, 0, 6, 1, 5, 0, 1, 0, 3, 6, 5, 1, 4, 5, 5, 9, 8, 6, 2, 5, 6, 2,
        1, 0, 2, 0, 1, 1, 6, 2, 3, 4, 6, 0, 9, 8, 8, 8, 0, 5, 1, 7, 0, 6, 1, 4,
        5, 6, 2, 8, 2, 0, 7, 4, 0, 7, 5, 2, 0, 6, 4, 2, 3, 8, 1, 3, 2, 3, 3, 4,
        2, 6, 7, 9, 2, 0, 0, 1, 6, 1, 8,

tensor([2, 2, 2, 7, 6, 2, 2, 5, 6, 8, 4, 2, 0, 0, 3, 2, 1, 4, 2, 2, 6, 2, 8, 3,
        9, 1, 5, 2, 5, 2, 0, 6, 8, 0, 3, 2, 3, 8, 8, 4, 9, 0, 4, 5, 4, 0, 6, 0,
        8, 6, 6, 8, 7, 1, 3, 2, 8, 7, 2, 2, 6, 5, 2, 2, 2, 2, 0, 2, 8, 8, 5, 8,
        2, 1, 2, 7, 3, 7, 8, 5, 0, 6, 1, 3, 1, 3, 3, 7, 0, 2, 1, 3, 8, 4, 8, 7,
        4, 1, 2, 6, 9, 1, 6, 3, 5, 9, 5, 9, 7, 4, 2, 0, 8, 0, 4, 8, 2, 3, 9, 1,
        8, 4, 2, 1, 3, 2, 8, 0, 8, 4, 8, 4, 1, 1, 0, 1, 5, 7, 1, 5, 5, 8, 4, 4,
        2, 1, 0, 8, 2, 3, 0, 2, 5, 4, 3, 8, 8, 3, 3, 6, 7, 2, 3, 8, 2, 3, 6, 6,
        3, 2, 0, 2, 8, 4, 2, 2, 8, 0, 0, 5, 0, 6, 8, 6, 0, 8, 7, 2, 1, 6, 2, 2,
        8, 6, 9, 1, 0, 3, 1, 3, 2, 5, 3, 4, 1, 6, 3, 7, 2, 6, 1, 4, 6, 6, 4, 7,
        1, 8, 5, 9, 0, 3, 9, 0, 8, 4, 0, 5, 7, 4, 4, 2, 5, 0, 3, 3, 1, 5, 0, 5,
        2, 8, 5, 0, 2, 2, 7, 9, 8, 4, 7, 2, 2, 2, 8, 8, 2, 0, 3, 6, 5, 7, 4, 3,
        2, 7, 2, 0, 2, 5, 7, 4, 9, 1, 7, 2, 2, 3, 8, 8, 4, 0, 1, 3, 7, 8, 3, 8,
        1, 2, 2, 0, 2, 7, 3, 2, 9, 3, 5,

In [111]:
for i in range(epochs):
    for batch, labels in data_loader:
        
        if using_cuda:
            x = Variable(batch.cuda())
            y = Variable(labels.cuda())
        else:
            x = Variable(batch)
            y = Variable(labels)
        optimizer.zero_grad()
        x = x.view(5,100,256)
        output = net(x)
        output = output.view(500,10)
        print(output.size(),y.flatten().size())
        loss = criterion(output, y.flatten())
        loss.backward()
        optimizer.step()
        
    n_true = 0
    for batch, labels in data_loader_test:
        if using_cuda:
            output = net(Variable(batch.cuda()))
        else:
            output = net(Variable(batch))
        
        _, predicted = torch.max(output.data, 1)
        if using_cuda:
            y_predicted = predicted.cpu().numpy()
        else:
            y_predicted = predicted.numpy()
        print(predicted,labels)
        n_true += np.sum(y_predicted == labels.numpy())

    total = 1000
    accuracy = 100 * n_true / total
    print('epoch: {0}, accuracy: {1}'.format(i, (int)(accuracy)))
    accuracies.append(accuracy)

print(accuracies)       

torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([0, 0, 0, 0, 0, 0, 9, 8, 8, 8, 0, 8, 0, 8, 8, 0, 0, 0, 0, 0, 0, 0, 2, 0,
        8, 8, 0, 0, 3, 8, 0, 0, 8, 8, 0, 8, 0, 0, 0, 8, 8, 8, 0, 0, 0, 0, 8, 0,
        8, 8, 8, 8, 0, 0, 8, 0, 0, 0, 0, 8, 0, 8, 8, 0, 8, 8, 8, 8, 8, 0, 0, 7,
        0, 0, 8, 0, 8, 7, 0, 8, 0, 8, 8, 8, 8, 8, 8, 0, 8, 0, 0, 8, 8, 0, 0, 8,
        0, 8, 8, 8, 8, 0, 0, 8, 0, 8, 8, 8, 8, 8, 0, 7, 8, 8, 0, 8, 0, 0, 0, 0,
        0, 0, 8, 8, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 8, 0, 8, 0, 0, 0, 0, 0,
        0, 0, 8, 0, 0, 0, 0, 8, 8, 8, 7, 0, 8, 0, 0, 8, 8, 0, 8, 0, 0, 0, 8, 8,
        0, 0, 8, 0, 0, 0, 8, 8, 0, 2, 0,

torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([9, 8, 0, 0, 0, 0, 0, 8, 2, 0, 8, 0, 8, 0, 8, 2, 8, 8, 2, 0, 9, 3, 0, 8,
        0, 0, 0, 2, 2, 8, 8, 0, 0, 8, 0, 0, 4, 3, 0, 0, 0, 0, 8, 0, 0, 0, 0, 8,
        0, 9, 8, 0, 8, 8, 8, 8, 8, 8, 0, 0, 3, 2, 0, 3, 0, 8, 0, 8, 3, 8, 8, 0,
        2, 0, 0, 0, 0, 8, 8, 8, 8, 0, 0, 8, 8, 2, 0, 8, 2, 3, 8, 8, 8, 0, 0, 0,
        0, 8, 8, 4, 8, 0, 3, 2, 0, 0, 0, 8, 8, 8, 2, 0, 0, 8, 0, 8, 8, 0, 0, 2,
        2, 8, 8, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 8, 8, 0, 8, 8,
        2, 0, 2, 8, 0, 8, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 8, 0, 0, 8, 0, 0, 8, 8,
        0, 0, 0, 8, 0, 0, 8, 0, 3, 0, 0, 8, 0, 0, 0, 0, 2, 0, 8, 2, 0, 0, 0, 0,


torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([3, 2, 3, 2, 4, 2, 8, 8, 8, 8, 8, 8, 8, 4, 8, 8, 8, 8, 9, 8, 8, 8, 3, 2,
        4, 0, 8, 8, 9, 8, 8, 8, 8, 0, 8, 8, 2, 8, 8, 8, 2, 8, 0, 0, 3, 0, 9, 8,
        8, 8, 9, 6, 8, 0, 9, 2, 8, 8, 8, 8, 8, 8, 9, 2, 2, 8, 8, 8, 0, 8, 0, 0,
        8, 8, 2, 8, 2, 9, 0, 2, 8, 3, 8, 8, 2, 8, 8, 8, 8, 8, 8, 8, 0, 2, 8, 0,
        4, 6, 8, 2, 8, 8, 8, 0, 8, 0, 4, 8, 8, 8, 8, 0, 2, 9, 8, 8, 8, 0, 0, 8,
        8, 0, 0, 0, 0, 8, 8, 8, 3, 3, 4, 8, 8, 8, 8, 8, 8, 8, 4, 8, 8, 8, 0, 8,
        0, 0, 8, 8, 0, 8, 0, 2, 4, 0, 8, 8, 4, 2, 0, 2, 8, 8, 0, 0, 2, 0, 4, 3,
        8, 6, 3, 6, 2, 0, 8, 3, 8, 0, 7,

torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([0, 8, 2, 0, 0, 8, 8, 8, 8, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 2, 8, 6, 2, 8,
        2, 2, 7, 8, 8, 8, 8, 4, 4, 7, 8, 8, 8, 0, 7, 7, 0, 8, 0, 0, 9, 4, 8, 4,
        8, 2, 2, 8, 8, 8, 0, 8, 9, 8, 8, 8, 8, 2, 8, 0, 8, 8, 8, 8, 2, 6, 8, 2,
        8, 8, 9, 8, 0, 2, 2, 8, 2, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 0, 7, 3, 2, 4,
        2, 2, 8, 4, 8, 2, 8, 8, 8, 8, 0, 8, 2, 8, 8, 8, 8, 8, 7, 8, 2, 8, 0, 8,
        8, 6, 0, 0, 2, 8, 8, 8, 3, 8, 0, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8, 7, 2, 8,
        8, 2, 8, 8, 0, 8, 8, 0, 0, 8, 0, 2, 8, 0, 0, 8, 8, 0, 9, 7, 7, 7, 8, 2,
        8, 8, 0, 8, 6, 8, 4, 8, 8, 8, 0, 3, 8, 8, 8, 0, 9, 0, 8, 8, 3, 8, 8, 9,
        8, 9, 8, 8, 8, 0, 9, 8, 4, 8, 8, 0, 8, 8, 8, 4, 8, 8, 8, 8, 8, 8, 8, 0,
        6, 8, 8, 8, 8, 9, 4, 8, 8, 8, 8,

torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([2, 8, 8, 8, 6, 7, 4, 4, 2, 8, 0, 0, 8, 8, 8, 8, 8, 2, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 0, 2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 8, 7, 8, 8, 8, 0, 8,
        8, 9, 8, 2, 7, 0, 8, 0, 2, 9, 4, 0, 8, 8, 8, 8, 9, 2, 7, 8, 0, 8, 8, 0,
        2, 2, 8, 2, 7, 0, 8, 8, 8, 8, 8, 8, 8, 8, 3, 8, 2, 8, 8, 8, 8, 9, 8, 8,
        8, 8, 0, 4, 8, 0, 8, 8, 6, 8, 8, 2, 8, 8, 6, 3, 8, 8, 0, 8, 8, 8, 8, 8,
        8, 0, 8, 6, 8, 8, 3, 0, 9, 8, 8, 8, 4, 8, 8, 2, 8, 3, 8, 2, 8, 8, 2, 3,
        8, 8, 0, 0, 2, 8, 8, 8, 2, 2, 8, 0, 8, 8, 2, 0, 7, 8, 8, 8, 8, 9, 0, 8,
        8, 0, 2, 8, 2, 8, 4, 8, 8, 8, 8, 8, 8, 2, 4, 9, 7, 2, 4, 8, 8, 9, 6, 8,
        2, 0, 8, 9, 8, 8, 8, 8, 0, 8, 8, 8, 7, 0, 0, 8, 8, 0, 8, 0, 8, 2, 8, 8,
        8, 8, 6, 2, 8, 8, 8, 2, 2, 9, 8,

...

torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([4, 8, 5, 4, 0, 8, 0, 1, 9, 6, 9, 4, 0, 0, 2, 2, 3, 4, 6, 2, 9, 2, 2, 9,
        5, 6, 1, 6, 8, 3, 8, 0, 4, 0, 3, 6, 9, 8, 4, 9, 3, 9, 2, 0, 1, 1, 1, 8,
        8, 0, 4, 3, 9, 4, 4, 3, 7, 8, 3, 8, 1, 1, 7, 0, 8, 8, 2, 9, 9, 4, 9, 8,
        1, 2, 8, 2, 8, 8, 3, 8, 3, 8, 8, 6, 2, 4, 3, 7, 8, 2, 4, 4, 7, 7, 4, 1,
        4, 8, 8, 9, 8, 5, 8, 9, 8, 8, 0, 1, 2, 8, 3, 9, 8, 1, 8, 8, 4, 4, 8, 4,
        4, 8, 4, 0, 5, 2, 2, 0, 7, 9, 8, 3, 9, 3, 8, 4, 2, 7, 6, 8, 7, 7, 4, 1,
        8, 8, 8, 8, 4, 0, 8, 8, 7, 5, 1, 0, 0, 8, 8, 5, 3, 8, 8, 2, 7, 0, 4, 8,
        6, 0, 1, 4, 9, 4, 1, 8, 0, 6, 0, 1, 2, 9, 2, 7, 1, 4, 4, 9, 9, 7, 1, 8,
        9, 8, 9, 0, 8, 8, 9, 0, 9, 1, 2,

torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([9, 8, 0, 3, 5, 9, 3, 4, 5, 9, 4, 8, 8, 4, 9, 0, 6, 9, 3, 2, 8, 0, 4, 8,
        4, 6, 0, 3, 4, 3, 6, 8, 0, 6, 1, 3, 9, 1, 8, 9, 8, 8, 8, 6, 0, 9, 1, 8,
        5, 8, 7, 8, 2, 2, 1, 8, 4, 3, 4, 8, 5, 4, 2, 2, 1, 8, 8, 2, 8, 3, 2, 3,
        0, 8, 4, 7, 2, 8, 0, 3, 3, 2, 5, 9, 4, 8, 6, 8, 6, 2, 9, 6, 1, 4, 2, 8,
        4, 7, 6, 1, 8, 3, 0, 4, 1, 2, 8, 4, 3, 4, 8, 8, 8, 9, 5, 2, 8, 1, 8, 9,
        0, 8, 3, 8, 6, 3, 0, 9, 7, 0, 6, 2, 0, 1, 5, 2, 7, 9, 9, 1, 7, 1, 6, 3,
        8, 6, 8, 0, 3, 2, 0, 9, 5, 8, 7, 8, 8, 2, 8, 0, 2, 9, 9, 9, 2, 1, 5, 0,
        4, 0, 0, 8, 6, 0, 5, 8, 5, 0, 8, 2, 8, 8, 8, 1, 9, 5, 9, 0, 8, 3, 2, 8,
        4, 0, 8, 0, 2, 9, 8, 1, 8, 5, 8, 1, 8, 2, 0, 4, 0, 3, 1, 1, 4, 4, 0, 8,
        1, 8, 8, 1, 4, 2, 9, 8, 1, 4, 2, 9, 0, 1, 4, 2, 8, 4, 2, 0, 7, 2, 8, 1,


        2, 9, 4, 4, 9, 3, 0, 4], device='cuda:0') tensor([4, 9, 8, 2, 3, 5, 3, 8, 5, 9, 7, 9, 2, 4, 5, 4, 7, 5, 6, 3, 4, 1, 7, 2,
        4, 7, 5, 0, 2, 6, 0, 0, 5, 3, 9, 2, 8, 3, 2, 3, 0, 0, 9, 3, 1, 4, 5, 2,
        6, 8, 3, 6, 5, 4, 5, 3, 4, 4, 7, 4, 9, 8, 6, 3, 2, 3, 5, 5, 6, 2, 0, 1,
        6, 2, 2, 7, 3, 1, 2, 9, 9, 7, 9, 4, 6, 7, 0, 5, 2, 3, 6, 1, 0, 4, 5, 3,
        8, 1, 4, 3, 0, 1, 8, 7, 1, 2, 8, 1, 9, 7, 0, 9, 2, 9, 6, 3, 4, 1, 1, 3,
        8, 4, 2, 7, 5, 0, 2, 1, 2, 5, 0, 7, 2, 1, 7, 1, 0, 5, 1, 8, 1, 4, 4, 1,
        9, 0, 6, 9, 1, 1, 0, 6, 1, 7, 8, 3, 0, 0, 1, 5, 0, 5, 1, 6, 0, 9, 8, 5,
        3, 8, 8, 8, 9, 7, 7, 6, 6, 3, 5, 9, 8, 4, 9, 1, 5, 7, 2, 6, 8, 2, 1, 4,
        2, 3, 8, 7, 1, 4, 3, 5, 1, 7, 2, 0, 7, 8, 3, 3, 4, 3, 5, 6, 2, 4, 0, 2,
        2, 7, 4, 5, 0, 6, 2, 7, 2, 3, 5, 4, 1, 3, 7, 8, 1, 9, 0, 7, 3, 0, 3, 5,
        6, 6, 4, 6, 5, 9, 5, 8, 0, 1, 8, 4, 4, 9, 4, 8, 1, 4, 5, 7, 8, 1, 5, 4,
        2, 6, 0, 9, 7, 8, 0, 0, 0, 6, 6, 7, 4, 7, 2, 6, 7, 6, 6, 6, 2,

torch.Size([500, 10]) torch.Size([500])
torch.Size([500, 10]) torch.Size([500])
tensor([8, 7, 7, 2, 8, 6, 5, 0, 0, 9, 8, 0, 2, 7, 8, 5, 8, 7, 4, 5, 8, 9, 4, 4,
        9, 8, 4, 6, 5, 8, 4, 1, 2, 2, 8, 4, 8, 9, 2, 4, 9, 6, 8, 4, 6, 6, 6, 0,
        3, 1, 6, 8, 3, 8, 4, 6, 1, 9, 9, 9, 2, 0, 8, 8, 3, 6, 6, 1, 3, 4, 6, 4,
        3, 8, 4, 6, 4, 9, 8, 7, 1, 5, 4, 5, 4, 0, 8, 2, 7, 2, 4, 3, 1, 1, 9, 0,
        7, 2, 7, 0, 9, 0, 9, 4, 6, 4, 8, 8, 0, 9, 9, 6, 4, 1, 5, 8, 4, 5, 8, 1,
        8, 6, 6, 1, 8, 7, 2, 8, 2, 2, 4, 8, 0, 8, 3, 4, 8, 4, 3, 4, 8, 4, 2, 7,
        1, 0, 9, 5, 3, 7, 8, 0, 2, 8, 9, 0, 1, 7, 7, 0, 3, 3, 1, 2, 2, 0, 4, 4,
        3, 9, 2, 0, 1, 2, 8, 4, 5, 2, 2, 4, 7, 6, 6, 0, 4, 1, 2, 8, 2, 0, 3, 1,
        0, 8, 5, 0, 9, 4, 8, 8, 9, 3, 4, 2, 0, 9, 0, 6, 2, 9, 7, 2, 0, 8, 6, 2,
        4, 9, 4, 6, 7, 5, 4, 4, 8, 8, 2, 8, 4, 4, 4, 4, 1, 2, 0, 4, 6, 9, 7, 4,
        4, 2, 8, 2, 8, 4, 4, 8, 4, 2, 6, 7, 6, 0, 8, 2, 0, 9, 9, 2, 8, 8, 4, 8,
        8, 4, 8, 6, 2, 8, 0, 7, 7, 6, 1,

KeyboardInterrupt: 

In [21]:
x = torch.randn([4,300,2])
y = torch.randn([4,2,10])

In [22]:
torch.mean(x,1)

tensor([[-0.0708,  0.0153],
        [ 0.0749, -0.1030],
        [-0.0154, -0.0261],
        [ 0.0333,  0.0414]])

In [24]:
torch.matmul(x,y).size()

torch.Size([4, 300, 10])

In [56]:
a = torch.ones([2,4,3])#.transpose(1,0)
b = torch.ones([2,3])*2.0
#nn.functional.log_softmax(torch.mm(a,b),dim=1)

In [52]:
a.size(),b

(torch.Size([4, 2, 3]), tensor([[2., 2., 2.],
         [2., 2., 2.]]))

In [30]:
a.flatten()

tensor([1., 1., 1., 1., 1., 1.])

In [58]:

c = torch.matmul(a[0],b[0]).view(-1,1)
d = torch.matmul(a[1],b[1]).view(-1,1)
c

tensor([[6.],
        [6.],
        [6.],
        [6.]])

In [60]:
torch.cat((c,d),1).size()

torch.Size([4, 2])

In [84]:
import random
s = torch.tensor([[[1,2,3,4,5],[6,7,8,9,10]]])

In [72]:
s = s.view(-1,10)

In [63]:
s.view(1,-1)

tensor([[1, 2, 3, 4, 5, 6]])

In [96]:
data_r_seet = list(range(5))
seet = data_r_seet
print(s[:,:,seet])
random.shuffle(data_r_seet)
seet = data_r_seet
print(seet)
print(s[:,:,seet])

tensor([[[ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10]]])
[0, 2, 1, 4, 3]
tensor([[[ 1,  3,  2,  5,  4],
         [ 6,  8,  7, 10,  9]]])


In [86]:
s.size()

torch.Size([1, 2, 5])